# UTS Data Mining

In [23]:
# data

company_name_list = [{'name': 'Company 1'},
          {'name': 'Company 2'},
          {'name': 'Company 3'}]

employee_name_list = [{'name': 'John Doe'},
          {'name': 'Tom Smith'},
          {'name': 'Andrew Sebastian'}]

company_detail_list = {
      'Company 1': {
          'name': 'Company 1',
          'domain': 'Retail',
          'clients': [
              {
                  'name': 'acme.inc',
                  'country': 'united states'
              },
              {
                  'name': 'Wayne.co',
                  'country': 'united states'
              }
          ]
      },
      'Company 2': {
          'name': 'Company 2',
          'domain': 'Construction',
          'clients': [
              {
                  'name': 'Tesla',
                  'country': 'united states'
              },
              {
                  'name': 'Japan Airlines',
                  'country': 'japan'
              },
              {
                  'name': 'Indofood',
                  'country': 'indonesia'
              }
          ]
      },
      'Company 3': {
          'name': 'Company 3',
          'domain': 'Healthcare',
          'clients': [
              {
                  'name': 'Petronas',
                  'country': 'malaysia'
              },
              {
                  'name': 'VW Group',
                  'country': 'germany'
              },
              {
                  'name': 'IBM',
                  'country': 'united states'
              },
              {
                  'name': 'Mitsubishi',
                  'country': 'japan'
              }
          ]
      }
  }

employee_detail_list = {
      'John Doe': {
          'name': 'EMP-0001',
          'first_name': 'John',
          'last_name': 'Doe',
          'full_name': 'John Doe',
          'company': 'Company 1'
      },
      'Tom Smith': {
          'name': 'EMP-0002',
          'first_name': 'Tom',
          'last_name': 'Smith',
          'full_name': 'Tom Smith',
          'company': 'Company 2'
      },
      'Andrew Sebastian': {
          'name': 'EMP-0003',
          'first_name': 'Andrew',
          'last_name': 'Sebastian',
          'full_name': 'Andrew Sebastian',
          'company': 'Company 2'
      },
  }

In [35]:
import json

# 1

### List semua Company yang di urutkan terbalik dari Domain nya

In [31]:
def sort_company():
    
    sorted_companies = sorted(company_detail_list.values(), key=lambda x: x['domain'], reverse=True)
    result = [{"name": company['name'], "domain": company['domain']} for company in sorted_companies]
    return result

sorted_companies = sort_company()
print(json.dumps(sorted_companies, indent=4))

[
    {
        "name": "Company 1",
        "domain": "Retail"
    },
    {
        "name": "Company 3",
        "domain": "Healthcare"
    },
    {
        "name": "Company 2",
        "domain": "Construction"
    }
]


# 2

### List semua nilai atau value dari Domain

In [34]:
company_names = [company['name'] for company in company_name_list]

def get_company_domain():
    for company_name in company_names:
        company_details = company_detail_list.get(company_name)
        if company_details:
            domain = company_details['domain']
            num_clients = len(company_details['clients'])
            print(f"{company_name}: {domain}, relation: {num_clients} clients")

get_company_domain()

Company 1: Retail, relation: 2 clients
Company 2: Construction, relation: 3 clients
Company 3: Healthcare, relation: 4 clients


# 3

### List Pegawai perusahaan beserta nama perusahaan nya

In [30]:
def get_employees():
    employees_with_domains = []
    for employee_name, employee_info in employee_detail_list.items():
        company_name = employee_info['company']
        domain = company_detail_list[company_name]['domain']
        employee_data = {
            "full_name": employee_info['full_name'],
            "company": company_name,
            "domain": domain
        }
        employees_with_domains.append(employee_data)
    return employees_with_domains

employees_with_domains = get_employees()
print(json.dumps(employees_with_domains, indent=4))

[
    {
        "full_name": "John Doe",
        "company": "Company 1",
        "domain": "Retail"
    },
    {
        "full_name": "Tom Smith",
        "company": "Company 2",
        "domain": "Construction"
    },
    {
        "full_name": "Andrew Sebastian",
        "company": "Company 2",
        "domain": "Construction"
    }
]


# 4

### List perusahaan dan pegawai yang bekerja di perusahaan tersebut

In [25]:
def get_employees_by_company():
    companies_with_employees = []
    for company_info in company_name_list:
        company_name = company_info['name']
        employees = [employee_info['full_name'] for employee_info in employee_detail_list.values() if employee_info['company'] == company_name]
        company_data = {
            "company": company_name,
            "employees": employees
        }
        companies_with_employees.append(company_data)
    return companies_with_employees

companies_with_employees = get_employees_by_company()
print(json.dumps(companies_with_employees, indent=4))

[
    {
        "company": "Company 1",
        "employees": [
            "John Doe"
        ]
    },
    {
        "company": "Company 2",
        "employees": [
            "Tom Smith",
            "Andrew Sebastian"
        ]
    },
    {
        "company": "Company 3",
        "employees": []
    }
]


# Soal Pre-Proccessing

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [77]:
data_startup = pd.read_csv('50_Startups.csv')

In [78]:
data_startup.sample(7)

R&D Spend  Administration  Marketing Spend       State     Profit
28   66051.52       182645.56        118148.20     Florida  103282.38
20   76253.86       113867.30        298664.47  California  118474.03
36   28663.76       127056.21        201126.82     Florida   90708.19
3   144372.41       118671.85        383199.62    New York  182901.99
15  114523.61       122616.84        261776.23    New York  129917.04
25   64664.71       139553.16        137962.62  California  107404.34
30   61994.48       115641.28         91131.24     Florida   99937.59

### 1. Analisa data kosong / NaN dan isi dengan 'mean'

In [92]:
X = data_startup.iloc[:, 0:5].values
X

array([[165349.2, 136897.8, 471784.1, 'New York', 192261.83],
       [162597.7, 151377.59, 443898.53, 'California', 191792.06],
       [153441.51, 101145.55, 407934.54, 'Florida', 191050.39],
       [144372.41, 118671.85, 383199.62, 'New York', 182901.99],
       [142107.34, 91391.77, 366168.42, 'Florida', 166187.94],
       [131876.9, 99814.71, 362861.36, 'New York', 156991.12],
       [134615.46, 147198.87, 127716.82, 'California', 156122.51],
       [130298.13, nan, 323876.68, 'Florida', 155752.6],
       [120542.52, 148718.95, 311613.29, 'New York', 152211.77],
       [123334.88, 108679.17, 304981.62, 'California', 149759.96],
       [nan, 110594.11, 229160.95, 'Florida', 146121.95],
       [100671.96, 91790.61, 249744.55, 'California', 144259.4],
       [93863.75, 127320.38, 249839.44, 'Florida', 141585.52],
       [91992.39, 135495.07, 252664.93, 'California', nan],
       [119943.24, 156547.42, 256512.92, 'Florida', 132602.65],
       [114523.61, 122616.84, 261776.23, 'New York'

In [93]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, [0, 1, 2, 4]])
X[:, [0, 1, 2, 4]] = imputer.transform(X[:, [0, 1, 2, 4]])

X

array([[165349.2, 136897.8, 471784.1, 'New York', 192261.83],
       [162597.7, 151377.59, 443898.53, 'California', 191792.06],
       [153441.51, 101145.55, 407934.54, 'Florida', 191050.39],
       [144372.41, 118671.85, 383199.62, 'New York', 182901.99],
       [142107.34, 91391.77, 366168.42, 'Florida', 166187.94],
       [131876.9, 99814.71, 362861.36, 'New York', 156991.12],
       [134615.46, 147198.87, 127716.82, 'California', 156122.51],
       [130298.13, 122790.15829787235, 323876.68, 'Florida', 155752.6],
       [120542.52, 148718.95, 311613.29, 'New York', 152211.77],
       [123334.88, 108679.17, 304981.62, 'California', 149759.96],
       [76788.862, 110594.11, 229160.95, 'Florida', 146121.95],
       [100671.96, 91790.61, 249744.55, 'California', 144259.4],
       [93863.75, 127320.38, 249839.44, 'Florida', 141585.52],
       [91992.39, 135495.07, 252664.93, 'California', 111850.85520833335],
       [119943.24, 156547.42, 256512.92, 'Florida', 132602.65],
       [114523.

## 2 OneHotEncoder pada field 'State'

In [94]:
label_encoder = LabelEncoder()

X[:, 3] = label_encoder.fit_transform(X[:, 3])
X

array([[165349.2, 136897.8, 471784.1, 2, 192261.83],
       [162597.7, 151377.59, 443898.53, 0, 191792.06],
       [153441.51, 101145.55, 407934.54, 1, 191050.39],
       [144372.41, 118671.85, 383199.62, 2, 182901.99],
       [142107.34, 91391.77, 366168.42, 1, 166187.94],
       [131876.9, 99814.71, 362861.36, 2, 156991.12],
       [134615.46, 147198.87, 127716.82, 0, 156122.51],
       [130298.13, 122790.15829787235, 323876.68, 1, 155752.6],
       [120542.52, 148718.95, 311613.29, 2, 152211.77],
       [123334.88, 108679.17, 304981.62, 0, 149759.96],
       [76788.862, 110594.11, 229160.95, 1, 146121.95],
       [100671.96, 91790.61, 249744.55, 0, 144259.4],
       [93863.75, 127320.38, 249839.44, 1, 141585.52],
       [91992.39, 135495.07, 252664.93, 0, 111850.85520833335],
       [119943.24, 156547.42, 256512.92, 1, 132602.65],
       [114523.61, 122616.84, 261776.23, 2, 129917.04],
       [78013.11, 121597.55, 264346.06, 0, 126992.93],
       [94657.16, 145077.58, 282574.31, 2, 

In [95]:
one_hot_encoder = OneHotEncoder(categories=X[3])
ct = ColumnTransformer([('country encoding', OneHotEncoder(), [3])], remainder='passthrough')
X = ct.fit_transform(X)
df = pd.DataFrame(X)
df

0    1    2          3              4              5              6
0   0.0  0.0  1.0   165349.2       136897.8       471784.1      192261.83
1   1.0  0.0  0.0   162597.7      151377.59      443898.53      191792.06
2   0.0  1.0  0.0  153441.51      101145.55      407934.54      191050.39
3   0.0  0.0  1.0  144372.41      118671.85      383199.62      182901.99
4   0.0  1.0  0.0  142107.34       91391.77      366168.42      166187.94
5   0.0  0.0  1.0   131876.9       99814.71      362861.36      156991.12
6   1.0  0.0  0.0  134615.46      147198.87      127716.82      156122.51
7   0.0  1.0  0.0  130298.13  122790.158298      323876.68       155752.6
8   0.0  0.0  1.0  120542.52      148718.95      311613.29      152211.77
9   1.0  0.0  0.0  123334.88      108679.17      304981.62      149759.96
10  0.0  1.0  0.0  76788.862      110594.11      229160.95      146121.95
11  1.0  0.0  0.0  100671.96       91790.61      249744.55       144259.4
12  0.0  1.0  0.0   93863.75      127320.38      249839.44      141585.52
13  1.0  0.0  0.0   91992.39      135495.07      252664.93  111850.855208
14  0.0  1.0  0.0  119943.24      156547.42      256512.92      132602.65
15  0.0  0.0  1.0  114523.61      122616.84      261776.23      129917.04
16  1.0  0.0  0.0   78013.11      121597.55      264346.06      126992.93
17  0.0  0.0  1.0   94657.16      145077.58      282574.31      125370.37
18  0.0  1.0  0.0   91749.16      114175.79      294919.57       124266.9
19  0.0  0.0  1.0    86419.7      153514.11  223582.690222      122776.86
20  1.0  0.0  0.0   76253.86       113867.3      298664.47      118474.03
21  0.0  0.0  1.0   78389.47      153773.43      299737.29      111313.02
22  0.0  1.0  0.0   73994.56      122782.75      303319.26      110352.25
23  0.0  1.0  0.0  76788.862      105751.03  223582.690222      108733.99
24  0.0  0.0  1.0   77044.01  122790.158298      140574.81      108552.04
25  1.0  0.0  0.0   64664.71      139553.16      137962.62      107404.34
26  0.0  1.0  0.0   75328.87      144135.98      134050.07      105733.54
27  0.0  0.0  1.0    72107.6      127864.55      353183.81      105008.31
28  0.0  1.0  0.0   66051.52      182645.56       118148.2      103282.38
29  0.0  0.0  1.0   65605.48      153032.06      107138.38      101004.64
30  0.0  1.0  0.0   61994.48      115641.28       91131.24       99937.59
31  0.0  0.0  1.0  76788.862      152701.92       88218.23  111850.855208
32  1.0  0.0  0.0   63408.86      129219.61       46085.25       97427.84
33  0.0  1.0  0.0   55493.95      103057.49      214634.81       96778.92
34  1.0  0.0  0.0   46426.07      157693.92      210797.67        96712.8
35  0.0  0.0  1.0   46014.02       85047.44      205517.64       96479.51
36  0.0  1.0  0.0   28663.76      127056.21      201126.82       90708.19
37  1.0  0.0  0.0   44069.95  122790.158298      197029.42       89949.14
38  0.0  0.0  1.0   20229.59       65947.93  223582.690222       81229.06
39  1.0  0.0  0.0   38558.51       82982.09       174999.3       81005.76
40  1.0  0.0  0.0   28754.33      118546.05      172795.67       78239.91
41  0.0  1.0  0.0   27892.92       84710.77      164470.71       77798.83
42  1.0  0.0  0.0   23640.93       96189.63      148001.11       71498.49
43  0.0  0.0  1.0   15505.73       127382.3       35534.17       69758.98
44  1.0  0.0  0.0   22177.74      154806.14       28334.72       65200.33
45  0.0  0.0  1.0    1000.23      124153.04        1903.93       64926.08
46  0.0  1.0  0.0    1315.46      115816.21      297114.46       49490.75
47  1.0  0.0  0.0  76788.862      135426.92  223582.690222       42559.73
48  0.0  0.0  1.0     542.05       51743.15  223582.690222       35673.41
49  1.0  0.0  0.0  76788.862       116983.8       45173.06        14681.4

## 3 Menambahkan field baru 'Tax'

In [99]:
new_column_values = (X[:, 1] + X[:, 2] + X[:, 4]) * 0.05

data_startup['Tax'] = new_column_values
data_startup.sample(7)

R&D Spend  Administration  Marketing Spend       State     Profit  \
20   76253.86       113867.30        298664.47  California  118474.03   
24   77044.01             NaN        140574.81    New York  108552.04   
13   91992.39       135495.07        252664.93  California        NaN   
9   123334.88       108679.17        304981.62  California  149759.96   
19   86419.70       153514.11              NaN    New York  122776.86   
39   38558.51        82982.09        174999.30  California   81005.76   
4   142107.34        91391.77        366168.42     Florida  166187.94   

            Tax  
20     5693.365  
24  6139.557915  
13    6774.7535  
9     5433.9585  
19    7675.7555  
39    4149.1045  
4     4569.6385

## 4 standardScaler pada tiap tiap kolom / field

In [87]:
sc = StandardScaler()
X = sc.fit_transform(X)
X

array([[-7.17740563e-01, -6.85994341e-01,  1.39326109e+00,
         2.64884988e+00, -2.72266385e-02,  2.88675135e+00,
         2.72222222e+00],
       [ 1.39326109e+00, -6.85994341e-01, -7.17740563e-01,
         2.64884988e+00,  1.33410529e+00,  2.88675135e+00,
         2.72222222e+00],
       [-7.17740563e-01,  1.45773797e+00, -7.17740563e-01,
         1.28346334e+00, -2.72266385e-02,  1.44337567e+00,
         1.33333333e+00],
       [-7.17740563e-01, -6.85994341e-01,  1.39326109e+00,
         1.28346334e+00, -2.72266385e-02,  1.44337567e+00,
         1.33333333e+00],
       [-7.17740563e-01,  1.45773797e+00, -7.17740563e-01,
         1.28346334e+00, -1.38855856e+00,  1.44337567e+00,
         1.33333333e+00],
       [-7.17740563e-01, -6.85994341e-01,  1.39326109e+00,
         1.28346334e+00, -2.72266385e-02,  1.44337567e+00,
         1.33333333e+00],
       [ 1.39326109e+00, -6.85994341e-01, -7.17740563e-01,
         1.28346334e+00, -2.72266385e-02, -1.33226763e-17,
         1.3333333